## LangGraph Quickstart

In [ ]:
# %%capture --no-stderr
# %pip install -U langgraph langsmith langchain_anthropic

In [ ]:
# %%capture --no-stderr
# %pip install -U langchain_community langchain_anthropic langchain_experimental matplotlib langgraph

## Test

In [1]:
from typing import TypedDict, Annotated, Optional, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import StateGraph, END
from langchain_core.tools import tool
from langchain_core.messages import ToolCall, ToolMessage
from langchain_openai import ChatOpenAI
from serpapi import GoogleSearch

from functools import partial

import os
from dotenv import load_dotenv



In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPAPI_KEY = os.getenv("SERPAPI_KEY")

In [3]:
class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]
    year: Optional[str]
    quarter: Optional[List]

In [ ]:


# search = GoogleSearch({
#         **serpapi_params,
#         "q": "what are the NVIDIA Risk factors",
#         "num": 3
#     })
# results = search.get_dict()["organic_results"]

In [ ]:
# contexts = "\n---\n".join(
#     ["\n".join([x["title"], x["snippet"], x["link"]]) for x in results])

In [ ]:
# print(contexts)

Nvidia (NVDA) Stock Risk Analysis
Risk Factors Full Breakdown - Total Risks 23 ; Economy & Political Environment1 | 4.3% ; International Operations1 | 4.3% ; Natural and Human Disruptions2 | 8.7%.
https://www.tipranks.com/stocks/nvda/risk-factors
---
4 Risks Nvidia Investors Should Consider as They ...
Returning to Nvidia, there are four risks that investors need to watch out for right now, even as they celebrate the stock split.
https://www.nasdaq.com/articles/4-risks-nvidia-investors-should-consider-they-celebrate-nvda-stock-split
---
What are the real risks of NVDA? : r/NVDA_Stock
Biggest risks to NVDA are 1) big tech decides enough spend is enough, 2) big tech succeeds in building their own good enough chips, and most ...
https://www.reddit.com/r/NVDA_Stock/comments/1eioqk4/what_are_the_real_risks_of_nvda/


In [5]:
# @tool
# def human_assistance(query: str) -> str:
#     """Request assistance from a human."""
#     human_response = interrupt({"query": query})
#     return human_response["data"]


@tool("web_search")
def web_search(query: str):
    """Finds general knowledge information using Google search. Can also be used
    to augment more 'general' knowledge to a previous specialist query."""
    serpapi_params = {
        "api_key": SERPAPI_KEY,  # <-- Add your SerpAPI key
        "engine": "google",  # Specifies Google Search Engine
    }

    search = GoogleSearch({
        **serpapi_params,
        "q": query,
        "num": 5
    })
    results = search.get_dict()["organic_results"]
    contexts = "\n---\n".join(
        ["\n".join([x["title"], x["snippet"], x["link"]]) for x in results]
    )
    return contexts

In [6]:
@tool("final_answer")
def final_answer(
    introduction: str,
    research_steps: str,
    main_body: str,
    conclusion: str,
    sources: str
):
    """Returns a natural language response to the user in the form of a research
    report. There are several sections to this report, those are:
    - `introduction`: a short paragraph introducing the user's question and the
    topic we are researching.
    - `research_steps`: a few bullet points explaining the steps that were taken
    to research your report.
    - `main_body`: this is where the bulk of high quality and concise
    information that answers the user's question belongs. It is 3-4 paragraphs
    long in length.
    - `conclusion`: this is a short single paragraph conclusion providing a
    concise but sophisticated view on what was found.
    - `sources`: a bulletpoint list provided detailed sources for all information
    referenced during the research process
    """
    if type(research_steps) is list:
        research_steps = "\n".join([f"- {r}" for r in research_steps])
    if type(sources) is list:
        sources = "\n".join([f"- {s}" for s in sources])
    return ""

In [ ]:
# system_prompt = """You are the oracle, the great AI decision maker.
# Given the user's query you must decide what to do with it based on the
# list of tools provided to you.

# If you see that a tool has been used (in the scratchpad) with a particular
# query, do NOT use that same tool with the same query again. Also, do NOT use
# any tool more than twice (ie, if the tool appears in the scratchpad twice, do
# not use it again).

# You should aim to collect information from a diverse range of sources before
# providing the answer to the user. Once you have collected plenty of information
# to answer the user's question (stored in the scratchpad) use the final_answer
# tool."""

# prompt = ChatPromptTemplate.from_messages([
#     ("system", system_prompt),
#     MessagesPlaceholder(variable_name="chat_history"),
#     ("user", "{input}"),
#     ("assistant", "scratchpad: {scratchpad}"),
# ])

In [ ]:
# from langchain_core.messages import ToolCall, ToolMessage
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     model="gpt-4o-mini",
#     openai_api_key=os.environ["OPENAI_API_KEY"],
#     temperature=0
# )

# tools=[
#     web_search,
#     final_answer
# ]

# # define a function to transform intermediate_steps from list
# # of AgentAction to scratchpad string
# def create_scratchpad(intermediate_steps: list[AgentAction]):
#     research_steps = []
#     for i, action in enumerate(intermediate_steps):
#         if action.log != "TBD":
#             # this was the ToolExecution
#             research_steps.append(
#                 f"Tool: {action.tool}, input: {action.tool_input}\n"
#                 f"Output: {action.log}"
#             )
#     return "\n---\n".join(research_steps)

# oracle = (
#     {
#         "input": lambda x: x["input"],
#         "chat_history": lambda x: x["chat_history"],
#         "scratchpad": lambda x: create_scratchpad(
#             intermediate_steps=x["intermediate_steps"]
#         ),
#     }
#     | prompt
#     | llm.bind_tools(tools, tool_choice="any")
# )

In [7]:
def init_research_agent(tool_keys, year=None, quarter=None):
    tool_str_to_func = {
            "web_search": web_search,
            "final_answer": final_answer
        }
    
    tools = [final_answer]
    for val in tool_keys:
        tools.append(tool_str_to_func[val])

    ## Designing Agent Features and Prompt ##
    system_prompt = f"""You are the oracle, the great AI decision maker.
    Given the user's query you must decide what to do with it based on the
    list of tools provided to you.

    Context:
    - Year: {year or 'Not specified'}
    - Quarter: {quarter or 'Not specified'}

    If you see that a tool has been used (in the scratchpad) with a particular
    query, do NOT use that same tool with the same query again. Also, do NOT use
    any tool more than twice (ie, if the tool appears in the scratchpad twice, do
    not use it again).

    You should aim to collect information from a diverse range of sources before
    providing the answer to the user. Once you have collected plenty of information
    to answer the user's question (stored in the scratchpad) use the final_answer
    tool."""

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        ("assistant", "scratchpad: {scratchpad}"),
    ])

    llm = ChatOpenAI(
        model="gpt-4o-mini",
        openai_api_key=os.environ["OPENAI_API_KEY"],
        temperature=0
    )

    def create_scratchpad(intermediate_steps: list[AgentAction]):
        research_steps = []
        for i, action in enumerate(intermediate_steps):
            if action.log != "TBD":
                # this was the ToolExecution
                research_steps.append(
                    f"Tool: {action.tool}, input: {action.tool_input}\n"
                    f"Output: {action.log}"
                )
        return "\n---\n".join(research_steps)

    oracle = (
        {
            "input": lambda x: x["input"],
            "chat_history": lambda x: x["chat_history"],
            "scratchpad": lambda x: create_scratchpad(
                intermediate_steps=x["intermediate_steps"]
            ),
        }
        | prompt
        | llm.bind_tools(tools, tool_choice="any")
    )
    return oracle


In [8]:
def run_oracle(state: AgentState, oracle):
    print("run_oracle")
    print(f"intermediate_steps: {state['intermediate_steps']}")
    out = oracle.invoke(state)
    tool_name = out.tool_calls[0]["name"]
    tool_args = out.tool_calls[0]["args"]
    action_out = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log="TBD"
    )
    return {
        **state,
        "intermediate_steps": [action_out]
    }

def router(state: AgentState):
    # return the tool name to use
    if isinstance(state["intermediate_steps"], list):
        return state["intermediate_steps"][-1].tool
    else:
        # if we output bad format go to final answer
        print("Router invalid format")
        return "final_answer"
    


def run_tool(state: AgentState):
    tool_str_to_func = {
        "web_search": web_search,
        "final_answer": final_answer
    }
    # use this as helper function so we repeat less code
    tool_name = state["intermediate_steps"][-1].tool
    tool_args = state["intermediate_steps"][-1].tool_input

    if tool_name in ["vector_search"]:
        tool_args = {
            **tool_args,
            "year": state.get("year"),
            "quarter": state.get("quarter")
        }
    print(f"{tool_name}.invoke(input={tool_args})")
    # run tool
    out = tool_str_to_func[tool_name].invoke(input=tool_args)
    action_out = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log=str(out)
    )
    return {
        **state,
        "intermediate_steps": [action_out]
    }


## Langraph - Designing the Graph
def create_graph(research_agent, year=None, quarter=None):
    tools=[
        web_search,
        final_answer
    ]

    graph = StateGraph(AgentState)  # Keep type definition here

    # Pass state to all functions that require it
    graph.add_node("oracle", partial(run_oracle, oracle=research_agent))
    graph.add_node("web_search", run_tool)
    graph.add_node("final_answer", run_tool)

    graph.set_entry_point("oracle")

    graph.add_conditional_edges(
        source="oracle",  # where in graph to start
        path=router,  # function to determine which node is called
    )

    # create edges from each tool back to the oracle
    for tool_obj in tools:
        if tool_obj.name != "final_answer":
            graph.add_edge(tool_obj.name, "oracle")

    # if anything goes to final answer, it must then move to END
    graph.add_edge("final_answer", END)

    runnable = graph.compile()
    return runnable

def run_agents(tool_keys, year=None, quarter=None):
    research_agent = init_research_agent(tool_keys, year, quarter)
    runnable = create_graph(research_agent, year, quarter)
    return runnable

# All Code start

In [1]:
from typing import TypedDict, Annotated, Optional, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import StateGraph, END
from langchain_core.tools import tool
from langchain_core.messages import ToolCall, ToolMessage
from langchain_openai import ChatOpenAI
from serpapi import GoogleSearch

from functools import partial

import os
from dotenv import load_dotenv

from pinecone_index import query_pinecone
from snowflake_agent import snowflake_query_agent

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPAPI_KEY = os.getenv("SERPAPI_KEY")


required_env_vars = {
    'SNOWFLAKE_ACCOUNT': os.getenv('SNOWFLAKE_ACCOUNT'),
    'SNOWFLAKE_USER': os.getenv('SNOWFLAKE_USER'),
    'SNOWFLAKE_PASSWORD': os.getenv('SNOWFLAKE_PASSWORD'),
    'SNOWFLAKE_ROLE': os.getenv('SNOWFLAKE_ROLE'),
    'SNOWFLAKE_DB': os.getenv('SNOWFLAKE_DB'),
    'SNOWFLAKE_WAREHOUSE': os.getenv('SNOWFLAKE_WAREHOUSE'),
    'SNOWFLAKE_SCHEMA': os.getenv('SNOWFLAKE_SCHEMA')
}


## Creating the Agent State ##
class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]
    year: Optional[str]
    quarter: Optional[List]



@tool("vector_search")
def vector_search(query: str, year: str = None, quarter: list = None):
    """Searches for the most relevant vector in the Pinecone index."""
    # query = "What is the revenue of Nvidia?"
    # year = "2025"
    # quarter = ['Q4', 'Q1']
    print("Reached Vector search 1")
    top_k = 10
    chunks = query_pinecone(query, top_k, year = year, quarter = quarter)
    contexts = "\n---\n".join(
        {chr(10).join([f'Chunk {i+1}: {chunk}' for i, chunk in enumerate(chunks)])}
    )
    return contexts

@tool("web_search")
def web_search(query: str):
    """Finds general knowledge information using Google search. Can also be used
    to augment more 'general' knowledge to a previous specialist query."""
    # Web Search Tool
    serpapi_params = {
        "api_key": SERPAPI_KEY,  # <-- Add your SerpAPI key
        "engine": "google",  # Specifies Google Search Engine
    }
    search = GoogleSearch({
        **serpapi_params,
        "q": query,
        "num": 5
    })
    results = search.get_dict()["organic_results"]
    contexts = "\n---\n".join(
        ["\n".join([x["title"], x["snippet"], x["link"]]) for x in results]
    )
    return contexts

# LangGraph tool integration
@tool("snowflake_query")
def snowflake_query(
    query: str = None, 
    analysis_type: str = 'financial_summary', 
    year: str = None, 
    quarter: List[str] = None
):
    """
    LangGraph compatible Snowflake query tool
    
    Args match snowflake_query_agent function
    """
    result = snowflake_query_agent(
        query=query, 
        analysis_type=analysis_type, 
        year=year, 
        quarter=quarter
    )
    
    # Convert Plotly figures to JSON for transmission
    if 'visualizations' in result:
        result['visualizations'] = {
            k: v.to_json() for k, v in result['visualizations'].items()
        }
        print(result['visualizations'])
    
    return str(result)



# Final Research Output Tool
# @tool("final_answer")
# def final_answer(
#     introduction: str,
#     research_steps: str,
#     main_body: str,
#     conclusion: str,
#     sources: list
# ):
#     """Returns a natural language response to the user in the form of a research
#     report. There are several sections to this report, those are:
#     - `introduction`: a short paragraph introducing the user's question and the
#     topic we are researching.
#     - `research_steps`: a few bullet points explaining the steps that were taken
#     to research your report.
#     - `main_body`: this is where the bulk of high quality and concise
#     information that answers the user's question belongs. It is 3-4 paragraphs
#     long in length.
#     - `conclusion`: this is a short single paragraph conclusion providing a
#     concise but sophisticated view on what was found.
#     - `sources`: a bulletpoint list provided detailed sources for all information
#     referenced during the research process
#     """
#     if type(research_steps) is list:
#         research_steps = "\n".join([f"- {r}" for r in research_steps])
#     if type(sources) is list:
#         sources = "\n".join([f"- {s}" for s in sources])
#     return ""

@tool("final_answer")
def final_answer(
    research_steps: str,
    historical_performance: str,
    financial_analysis: str,
    financial_visualizations: dict,
    industry_insights: str,
    sources: list
):
    """
    Returns a comprehensive research report combining data from all agents.
    
    Args:
    -research_steps: a few bullet points explaining the steps that were taken
#     to research your report. Mention the tool name and what step you took
    - historical_performance: Analysis from RAG/vector_search agent 
    - financial_analysis: Snowflake agent's financial summary and metrics
    - financial_visualizations: Plotly figures from Snowflake analysis
    - industry_insights: Real-time trends from web search agent
    - sources: List of all referenced sources
    
    Returns:
    Structured dictionary with complete research report components
    """
    if type(research_steps) is list:
        research_steps = "\n".join([f"- {r}" for r in research_steps])
    if isinstance(sources, list):
        sources = "\n".join([f"- {s}" for s in sources])
    
    report = {
        "research_steps": research_steps,
        "historical_performance": historical_performance,
        "financial_analysis": financial_analysis,
        "financial_visualizations": financial_visualizations,
        "industry_insights": industry_insights,
        "sources": sources
    }
    
    return report

def init_research_agent(tool_keys, year=None, quarter=None):
    tool_str_to_func = {
            "web_search": web_search,
            "vector_search": vector_search,
            "snowflake_query": snowflake_query,
            "final_answer": final_answer
        }
    
    tools = [final_answer]
    for val in tool_keys:
        tools.append(tool_str_to_func[val])

    ## Designing Agent Features and Prompt ##
    system_prompt = f"""You are NVIDIA research agent that has multiple tools available for research and NVIDIA information retrieval.
    Given the user's query you must decide what to do with it based on the
    list of tools provided to you.

    Context:
    - Year: {year or 'Not specified'}
    - Quarter: {quarter or 'Not specified'}

    Use all the Tools available at least once.
    If you see that a tool has been used (in the scratchpad) with a particular
    query, do NOT use that same tool with the same query again. Also, do NOT use
    any tool more than twice (ie, if the tool appears in the scratchpad twice, do
    not use it again).

    You should aim to collect information from a diverse range of sources regarding NVIDIA before
    providing the answer to the user. Once you have collected plenty of information
    to answer the user's question (stored in the scratchpad) use the final_answer
    tool."""

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        ("assistant", "scratchpad: {scratchpad}"),
    ])

    llm = ChatOpenAI(
        model="gpt-4o-mini",
        openai_api_key=os.environ["OPENAI_API_KEY"],
        temperature=0
    )

    def create_scratchpad(intermediate_steps: list[AgentAction]):
        research_steps = []
        for i, action in enumerate(intermediate_steps):
            if action.log != "TBD":
                # this was the ToolExecution
                research_steps.append(
                    f"Tool: {action.tool}, input: {action.tool_input}\n"
                    f"Output: {action.log}"
                )
        return "\n---\n".join(research_steps)

    oracle = (
        {
            "input": lambda x: x["input"],
            "chat_history": lambda x: x["chat_history"],
            "scratchpad": lambda x: create_scratchpad(
                intermediate_steps=x["intermediate_steps"]
            ),
        }
        | prompt
        | llm.bind_tools(tools, tool_choice="any")
    )
    return oracle

## Router and Parent Agent functions
def run_oracle(state: AgentState, oracle):
    print("run_oracle")
    print(f"intermediate_steps: {state['intermediate_steps']}")
    out = oracle.invoke(state)
    tool_name = out.tool_calls[0]["name"]
    tool_args = out.tool_calls[0]["args"]
    action_out = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log="TBD"
    )
    return {
        **state,
        "intermediate_steps": [action_out]
    }

def router(state: AgentState):
    # return the tool name to use
    if isinstance(state["intermediate_steps"], list):
        return state["intermediate_steps"][-1].tool
    else:
        # if we output bad format go to final answer
        print("Router invalid format")
        return "final_answer"
    


def run_tool(state: AgentState):
    tool_str_to_func = {
            "web_search": web_search,
            "vector_search": vector_search,
            "snowflake_query": snowflake_query,
            "final_answer": final_answer
        }
    
    # use this as helper function so we repeat less code
    tool_name = state["intermediate_steps"][-1].tool
    tool_args = state["intermediate_steps"][-1].tool_input

    if tool_name in ["vector_search"]:
        tool_args = {
            **tool_args,
            "year": state.get("year"),
            "quarter": state.get("quarter")
        }
    print(f"{tool_name}.invoke(input={tool_args})")
    # run tool
    out = tool_str_to_func[tool_name].invoke(input=tool_args)
    action_out = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log=str(out)
    )
    return {
        **state,
        "intermediate_steps": [action_out]
    }


## Langraph - Designing the Graph
def create_graph(research_agent, year=None, quarter=None):
    tools=[
        vector_search,
        snowflake_query,
        web_search,
        final_answer
    ]

    graph = StateGraph(AgentState)  # Keep type definition here

    # Pass state to all functions that require it
    graph.add_node("oracle", partial(run_oracle, oracle=research_agent))
    graph.add_node("web_search", run_tool)
    graph.add_node("vector_search", run_tool)
    graph.add_node("snowflake_query", run_tool)
    graph.add_node("final_answer", run_tool)

    graph.set_entry_point("oracle")

    graph.add_conditional_edges(
        source="oracle",
        path=router,
    )

    # create edges from each tool back to the oracle
    for tool_obj in tools:
        if tool_obj.name != "final_answer":
            graph.add_edge(tool_obj.name, "oracle")

    # if anything goes to final answer, it must then move to END
    graph.add_edge("final_answer", END)

    runnable = graph.compile()
    return runnable

def run_agents(tool_keys, year=None, quarter=None):
    research_agent = init_research_agent(tool_keys, year, quarter)
    runnable = create_graph(research_agent, year, quarter)
    return runnable


c:\Users\yohan\anaconda3\envs\DAMG_5\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tool_keys = ["snowflake_query", "final_answer"]
# tool_keys = ["final_answer"]

oracle = init_research_agent(tool_keys)

In [4]:
oracle

{
  input: RunnableLambda(...),
  chat_history: RunnableLambda(...),
  scratchpad: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['chat_history', 'input', 'scratchpad'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')

In [2]:
tool_keys = ["snowflake_query", "final_answer"]
year = "2025"
quarter = ["Q1", "Q3"]
runnable = run_agents(tool_keys, year, quarter)


In [3]:
out = runnable.invoke({ 
    "input": "What was relevant about Nvidia stocks in the year and quarter", 
    "chat_history": [], 
    "year": year, 
    "quarter": quarter 
})

run_oracle
intermediate_steps: []
snowflake_query.invoke(input={'year': '2025', 'quarter': ['Q1']})


c:\Users\yohan\OneDrive\Documents\Yohan\MASTERS\Classes\DAMG 7245\Assignments\Assignment-5\Part-1\DAMG7245_Assignment05_Part01\prototypes\snowflake_agent.py:229: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


{'revenue': '{"data":[{"hovertemplate":"DATE=%{x}\\u003cbr\\u003eTOTAL_REVENUE=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e","legendgroup":"","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"","orientation":"v","showlegend":false,"x":["2025-01-31T00:00:00"],"xaxis":"x","y":{"dtype":"f8","bdata":"AABAMjpiPkI="},"yaxis":"y","type":"scatter"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0

c:\Users\yohan\OneDrive\Documents\Yohan\MASTERS\Classes\DAMG 7245\Assignments\Assignment-5\Part-1\DAMG7245_Assignment05_Part01\prototypes\snowflake_agent.py:229: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



{'revenue': '{"data":[{"hovertemplate":"DATE=%{x}\\u003cbr\\u003eTOTAL_REVENUE=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e","legendgroup":"","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"","orientation":"v","showlegend":false,"x":[],"xaxis":"x","y":[],"yaxis":"y","type":"scatter"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a

In [4]:
out["intermediate_steps"][-1].tool_input

{}

intermediate_steps: [AgentAction(tool='web_search', tool_input={'query': 'investment growth trajectory analysis 2025'}, log='TBD'), AgentAction(tool='web_search', tool_input={'query': 'investment growth trajectory analysis 2025'}, log='Stock Market Outlook 2025: Can the Bull Run Persist?\nAfter two strong years for stocks, more muted gains are likely in 2025, with opportunities in U.S. stocks, growth and value.\nhttps://www.morganstanley.com/insights/articles/stock-market-outlook-2025\n---\n2025 Investment Directions: Navigating Market Trends\nExplore key investment directions for 2025 and make informed decisions with our analysis on U.S. growth, global equities, and strategic risk\nhttps://www.ishares.com/us/insights/investment-directions-year-ahead-2025\n---\nMapping Out the 2025 Investment Landscape Across Asset ...\nPrivate credit opportunities appear likely to persist in 2025—and possibly expand if the US economy avoids falling into a recession. So far, the ...\nhttps://www.alliancebernstein.com/us/en-us/investments/insights/investment-insights/mapping-out-the-2025-investment-landscape-across-asset-classes.html\n---\n2025 Investment Outlook\nInstitutional allocators face new, complex dynamics across the investment universe in 2025. Even as opportunities abound across the public ...\nhttps://www.pionline.com/investment-outlook2025\n---\n2025 Investment Outlook | BlackRock Investment Institute\nLooking ahead to 2025, we see persistent inflation pressures fueled by rising geopolitical fragmentation, plus big spending on the AI buildout ...\nhttps://www.blackrock.com/us/individual/insights/blackrock-investment-institute/outlook')]

# End

## Test end

In [19]:
inputs = {
    "input": "tell me something interesting about dogs",
    "chat_history": [],
    "intermediate_steps": [],
}
out = oracle.invoke(inputs)
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jWgXflmaLeR3VV1wMBXnQH52', 'function': {'arguments': '{"query":"interesting facts about dogs"}', 'name': 'web_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 612, 'total_tokens': 630, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bbfba58e46', 'id': 'chatcmpl-BFT3NVDR9qyij6PrjvGDkFM9xpB0T', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b067df04-50a6-4d5c-9689-1f2a6acf2248-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'interesting facts about dogs'}, 'id': 'call_jWgXflmaLeR3VV1wMBXnQH52', 'type': 'tool_call'}], usage_metadata={'input_tokens': 612, 'output_tokens': 18, 'total_tokens': 630, 'input_to

In [6]:
out.tool_calls[0]["name"]

AttributeError: 'AddableValuesDict' object has no attribute 'tool_calls'

In [21]:
out.tool_calls[0]["args"]

{'query': 'interesting facts about dogs'}

In [ ]:
# def run_oracle(state: list):
#     print("run_oracle")
#     print(f"intermediate_steps: {state['intermediate_steps']}")
#     out = oracle.invoke(state)
#     tool_name = out.tool_calls[0]["name"]
#     tool_args = out.tool_calls[0]["args"]
#     action_out = AgentAction(
#         tool=tool_name,
#         tool_input=tool_args,
#         log="TBD"
#     )
#     return {
#         "intermediate_steps": [action_out]
#     }

# def router(state: list):
#     # return the tool name to use
#     if isinstance(state["intermediate_steps"], list):
#         return state["intermediate_steps"][-1].tool
#     else:
#         # if we output bad format go to final answer
#         print("Router invalid format")
#         return "final_answer"

In [ ]:
# tool_str_to_func = {
#     "web_search": web_search,
#     "final_answer": final_answer
# }

# def run_tool(state: list):
#     # use this as helper function so we repeat less code
#     tool_name = state["intermediate_steps"][-1].tool
#     tool_args = state["intermediate_steps"][-1].tool_input
#     print(f"{tool_name}.invoke(input={tool_args})")
#     # run tool
#     out = tool_str_to_func[tool_name].invoke(input=tool_args)
#     action_out = AgentAction(
#         tool=tool_name,
#         tool_input=tool_args,
#         log=str(out)
#     )
#     return {"intermediate_steps": [action_out]}

In [ ]:
# from langgraph.graph import StateGraph, END

# graph = StateGraph(AgentState)

# graph.add_node("oracle", run_oracle)
# graph.add_node("web_search", run_tool)
# graph.add_node("final_answer", run_tool)

# graph.set_entry_point("oracle")

# graph.add_conditional_edges(
#     source="oracle",  # where in graph to start
#     path=router,  # function to determine which node is called
# )

# # create edges from each tool back to the oracle
# for tool_obj in tools:
#     if tool_obj.name != "final_answer":
#         graph.add_edge(tool_obj.name, "oracle")

# # if anything goes to final answer, it must then move to END
# graph.add_edge("final_answer", END)

# runnable = graph.compile()

In [39]:
# from IPython.display import Image

# Image(runnable.get_graph().draw_png())

In [40]:
out = runnable.invoke({
    "input": "tell me something interesting about dogs",
    "chat_history": [],
})

run_oracle
intermediate_steps: []
web_search.invoke(input={'query': 'interesting facts about dogs'})
run_oracle
intermediate_steps: [AgentAction(tool='web_search', tool_input={'query': 'interesting facts about dogs'}, log='TBD'), AgentAction(tool='web_search', tool_input={'query': 'interesting facts about dogs'}, log="30 Fun and Fascinating Dog Facts\nCheck out these 30 fun dog facts. 1. The Labrador Retriever has been on the AKC's top 10 most popular breeds list for longer than any other breed.\nhttps://www.akc.org/expert-advice/lifestyle/dog-facts/\n---\nInteresting Facts About Dogs\n– When a puppy is born, he is blind, deaf, and toothless. – All dogs, regardless of breed, are direct descendants of wolves and technically of the same species.\nhttps://www.mspca.org/pet_resources/interesting-facts-about-dogs/\n---\n10 Awesome Dog Facts\nDog facts! · 1. Dogs are the most popular pet on the planet! · 2. They evolved from a now-extinct species of wolf. · 3. They can learn over 100 words a

In [41]:
out["intermediate_steps"][-1].tool_input

{'introduction': 'Dogs are fascinating creatures that have been companions to humans for thousands of years. Their unique characteristics and behaviors make them one of the most beloved pets worldwide.',
 'research_steps': '1. Conducted a web search for interesting facts about dogs. 2. Reviewed multiple sources to gather diverse information. 3. Compiled notable facts and insights about dogs.',
 'main_body': 'Dogs are not only the most popular pets globally, but they also have a rich history and unique traits that set them apart from other animals. One interesting fact is that all dogs, regardless of their breed, are direct descendants of wolves, showcasing their evolutionary journey. This connection to wolves is evident in their social behaviors and pack mentality. \n\nAnother remarkable aspect of dogs is their ability to understand human emotions and gestures. Research indicates that dogs can learn over 100 words and can interpret human facial expressions, making them incredibly attun

In [ ]:
query = """        SELECT
            DATE,
            TOTAL_REVENUE,
            NET_INCOME,
            EBITDA,
            DILUTED_EPS
        FROM NVDA_FINANCIALS
         WHERE SUBSTRING(DATE, 1, 4) = '2023' AND ((SUBSTRING(DATE, 6, 2) IN ('01', '02', '03'))) ORDER BY DATE DESC LIMIT 100
         """

In [1]:
def format_result(result):
    # Initialize an empty list to collect parts of the string
    parts = []
    
    # Handle the 'data' key if present
    if 'data' in result and result['data']:
        parts.append("DATA:")
        # Loop through each dictionary in the list
        for i, item in enumerate(result['data'], start=1):
            parts.append(f"Record {i}:")
            # Loop through each key-value pair in the item (keys are dynamic)
            for key, value in item.items():
                parts.append(f"  {key.title()}: {value}")
            parts.append("")  # Add a blank line between records
    
    # Handle the 'summary' key if present
    if 'summary' in result:
        parts.append("SUMMARY:")
        # Remove extra whitespace if needed
        summary = result['summary'].strip()
        parts.append(summary)
        parts.append("")
    
    # Handle the 'query' key if present
    if 'query' in result:
        parts.append("QUERY:")
        # Remove extra whitespace if needed
        query = result['query'].strip()
        parts.append(query)
    
    # Join all parts into one string with newline characters
    return "\n".join(parts)

In [2]:
result_dict = {
    'data': [
        {
            'DATE': "2023-01-31 00:00:00", 
            'TOTAL_REVENUE': 26974000000.0, 
            'NET_INCOME': 4368000000.0, 
            'EBITDA': 5987000000.0, 
            'DILUTED_EPS': 0.17
        }
    ],
    'summary': '''
        Financial Summary:
        - Average Revenue: $26,974,000,000.00
        - Average Net Income: $4,368,000,000.00
        - Average Diluted EPS: $0.17
        ''',
    'query': '''
        SELECT 
            DATE, 
            TOTAL_REVENUE, 
            NET_INCOME, 
            EBITDA, 
            DILUTED_EPS
        FROM NVDA_FINANCIALS
        WHERE SUBSTRING(DATE, 1, 4) = '2023' 
          AND ((SUBSTRING(DATE, 6, 2) IN ('01', '02', '03')))
        ORDER BY DATE DESC LIMIT 100
        '''
}

In [3]:
formatted_string = format_result(result_dict)
print(formatted_string)

DATA:
Record 1:
  Date: 2023-01-31 00:00:00
  Total_Revenue: 26974000000.0
  Net_Income: 4368000000.0
  Ebitda: 5987000000.0
  Diluted_Eps: 0.17

SUMMARY:
Financial Summary:
        - Average Revenue: $26,974,000,000.00
        - Average Net Income: $4,368,000,000.00
        - Average Diluted EPS: $0.17

QUERY:
SELECT 
            DATE, 
            TOTAL_REVENUE, 
            NET_INCOME, 
            EBITDA, 
            DILUTED_EPS
        FROM NVDA_FINANCIALS
        WHERE SUBSTRING(DATE, 1, 4) = '2023' 
          AND ((SUBSTRING(DATE, 6, 2) IN ('01', '02', '03')))
        ORDER BY DATE DESC LIMIT 100
